In [ ]:
import scala.collection.mutable
import smile.plot._
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.math.distances.ContinuousDistance
import org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityScalar
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.clusterizables.EasyClusterizable
import org.apache.spark.storage.StorageLevel
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.hashing.LDLSH
import org.clustering4ever.spark.clustering.GradientAscent
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal

import scala.collection.mutable
import smile.plot._
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.math.distances.ContinuousDistance
import org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityScalar
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.clusterizables.EasyClusterizable
import org.apache.spark.storage.StorageLevel
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.hashing.LDLSH
import org.clustering4ever.spark.clustering.GradientAscent
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/DS1/DS1.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/DS1/DS1-labels.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2019-05-09 11:14:33--  http://www.clustering4ever.org/Datasets/DS1/DS1.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73786 (72K) [text/csv]
Saving to: ‘/tmp/DS1.csv.3’

     0K .......... .......... .......... .......... .......... 69%  238K 0s
    50K .......... .......... ..                              100%  102K=0.4s

2019-05-09 11:14:33 (169 KB/s) - ‘/tmp/DS1.csv.3’ saved [73786/73786]

:sh: Scheme missing.
--2019-05-09 11:14:33--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-05-09 11:14:33--  http://www.clustering4ever.org/Datasets/DS1/DS1-labels.csv
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 19315 (19K) [text/csv]
Saving to: ‘/tmp/DS1-labels.csv.3’


In [ ]:
type SeqType = mutable.ArrayBuffer[Double]
val path = "/tmp/aggregation.csv"
val data = scala.io.Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).zipWithIndex
  .map{ case (v, id) => EasyClusterizable(id.toLong, ScalarVector(v)) }.toSeq
val rdd = sc.parallelize(data)
val labelsPath = "/tmp/labels"

defined type alias SeqType
path: String = /tmp/aggregation.csv
data: Seq[org.clustering4ever.clusterizables.EasyClusterizable[None.type,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = Stream(EasyClusterizable(0,Vectorizable(None),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@6bc1d167,Vector()), ?)
rdd: org.apache.spark.rdd.RDD[org.clustering4ever.clusterizables.EasyClusterizable[None.type,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = ParallelCollectionRDD[0] at parallelize at <console>:86
labelsPath: String = /tmp/labels


## Gradient Ascent parameters

In [ ]:
val k = 80 // Main Gradient Ascent parameter, the k of knn gradient ascent
val k2 = 1 // Optional parameter which determine if studied point has to switch to its knn majority vote neighbor bucket
val epsilon = 0.01 // Stopping criteria for Gradient Ascent, every point have to moove less than epsilon in order to stop iteration
val maxNumberIter = 20 
val bucketLayers = 1 // Neighbor bucket layer take into account during Gradient Ascent, at least 1 is recommended for decent k values, the bigger it is the slower it become
val w = 1D // Don't observe any change by changing this value, let it to whatever you want
val bucketNumber = 8 // Approximatively DS_Size / 1000 for decent performance, the large buckets are the slower it becomes
val propConvStopIter = 1D // Supplementary stopping criteria, stop if propConvStopIter % of data have converged
val memoryExpensive = true // Choose for fast or slow computation at some memory cost (building of similarity matrix per bucket per iter or on the fly (more calculus))
val persistanceLVL: StorageLevel = StorageLevel.MEMORY_ONLY // Persistance Spark level 
val dim = 2 // Dim of your dataset
val lsh = LDLSH[SeqType](dim, w)

k: Int = 80
k2: Int = 1
epsilon: Double = 0.01
maxNumberIter: Int = 20
bucketLayers: Int = 1
w: Double = 1.0
bucketNumber: Int = 8
propConvStopIter: Double = 1.0
memoryExpensive: Boolean = true
persistanceLVL: org.apache.spark.storage.StorageLevel = StorageLevel(memory, deserialized, 1 replicas)
dim: Int = 2
lsh: org.clustering4ever.hashing.LDLSH[SeqType] = LDLSH(2,1.0)


## Epsilon proximity parameters

In [ ]:
// Main parameter epsilon can be approximate with average distance knn => "knn:88" or be expressed directly as a double "eps:3.67"
val epsilonEpsProx = "knn:40"
// val epsilonEpsProx = "eps:2.56"
val classicalMetric = Euclidean[SeqType](false)
val bucketNumber = 8 // Number of buckets in LSH, Approximatively DS_Size / 1000 for decent performance, the large buckets are the slower it becomes
val bucketLayers = 1 // Neighbor bucket layer take into account during Gradient Ascent, at least 1 is recommended for decent k values, the bigger it is the slower it become
val methodChoice = "bydot:1" // Fusionning cluster methods, at least X dot have to satisfy fusionning condition with "bydot:X", X=1 is greatly recommended
val divisionFactor = 1 // Experimental, let it to 1, it is supposed to speedup process on large datasets with bigger values but doesn't affect results
val cmin = 0 // Fusion clusters that have less than X elements with their closest cluster, it is in O(c²) with c number of clusters.

epsilonEpsProx: String = knn:40
classicalMetric: org.clustering4ever.math.distances.scalar.Euclidean[SeqType] = Euclidean distance without squared root applied
bucketNumber: Int = 8
bucketLayers: Int = 1
methodChoice: String = bydot:1
divisionFactor: Int = 1
cmin: Int = 0


In [ ]:
// Theses 3 params have to be here else there is some bugs..., it works more properly on full programm versions
val idOriginalVector = 8
val idToStoreGAVector = 0
val vm = VMapping[Int, ScalarVector[SeqType]]

val t0 = System.currentTimeMillis

val ascended = GradientAscent.train(rdd, k, epsilon, maxNumberIter, bucketNumber, bucketLayers, k2, propConvStopIter, memoryExpensive,
  persistanceLVL, idToStoreGAVector, true)


val updatedData = ascended.map( cz => cz.addAlternativeVector(idOriginalVector, cz.v).switchForExistingVectorization(idToStoreGAVector, vm) )

val model = EpsilonProximityScalar(
  epsilonEpsProx,
  methodChoice,
  classicalMetric,
  bucketNumber,
  lsh,
  cmin,
  bucketLayers,
  StorageLevel.MEMORY_ONLY,
  divisionFactor
).fit(updatedData)

val clusterizedRDD = model.obtainInputDataClustering(updatedData)

val predLabels = clusterizedRDD.map(_.clusterIDs.head).collect

val t1 = System.currentTimeMillis

val trueLabels = scala.io.Source.fromFile(labelsPath).getLines.map(_.toInt).toSeq.toArray
val targetAndPred = trueLabels.zip(predLabels)

(t1 - t0) / 1000D

idOriginalVector: Int = 8
idToStoreGAVector: Int = 0
vm: org.clustering4ever.shapeless.VMapping[Int,org.clustering4ever.vectors.ScalarVector[SeqType]] = org.clustering4ever.shapeless.VMapping@24ca2c10
t0: Long = 1557393278748
ascended: org.apache.spark.rdd.RDD[org.clustering4ever.clusterizables.EasyClusterizable[None.type,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = MapPartitionsRDD[32] at map at GradientAscent.scala:223
updatedData: org.apache.spark.rdd.RDD[org.clustering4ever.clusterizables.EasyClusterizable[None.type,org.clustering4ever.vectors.ScalarVector[SeqType]]] = MapPartitionsRDD[33] at map at <console>:118
model: org.clustering4ever.clustering.epsilonproximity.rdd.EpsilonProximityModelScalar[SeqType,org.clustering4ever.math.distan...

5.428

## Plot results if there are less than 25 clusters

In [ ]:
val preViz = clusterizedRDD.map( cz => (cz.clusterIDs.head, cz.v.vector.toArray) ).collect
val rawData = clusterizedRDD.map(_.vectorized.get(idOriginalVector)(vm).get.vector.toArray).collect
val rawDataConverged = preViz.map(_._2)
val clusterIDs = preViz.map(_._1)

preViz: Array[(Int, Array[Double])] = Array((5,Array(12.866249999999999, 25.054374999999997)), (5,Array(12.866249999999999, 25.054374999999997)), (5,Array(12.866249999999999, 25.054374999999997)), (5,Array(9.738749999999998, 25.246875)), (5,Array(9.738749999999998, 25.246875)), (5,Array(9.738749999999998, 25.246875)), (5,Array(9.738749999999998, 25.246875)), (5,Array(8.314375000000002, 24.136249999999997)), (5,Array(8.221875, 24.159374999999997)), (5,Array(8.221875, 24.159374999999997)), (5,Array(7.144374999999998, 22.196874999999995)), (6,Array(7.628749999999999, 15.703749999999996)), (5,Array(7.328749999999998, 21.938749999999995)), (6,Array(7.628749999999999, 15.703749999999996)), (5,Array(7.8125, 21.235625)), (5,Array(7.8125, 21.235625)), (5,Array(8.365, 23.965624999999996)), (5,Arr...

In [ ]:
plot(rawData, clusterIDs, '*', Palette.COLORS)
plot(rawDataConverged, clusterIDs, '*', Palette.COLORS)

res32: smile.plot.Window = Window(javax.swing.JFrame[frame2,460,37,1000x1000,invalid,layout=java.awt.BorderLayout,title=Smile Plot 4,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,5,25,990x970,invalid,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777673,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,0x0,invalid,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=9,maximumSize=,minimumSize=,preferredSize=])


Window(javax.swing.JFrame[frame2,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 4,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,37,1000x963,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x963,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

In [ ]:
val mei = new MultiExternalIndicesLocal(targetAndPred)

mei: org.clustering4ever.clustering.indices.MultiExternalIndicesLocal = MultiExternalIndicesLocal(WrappedArray((2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,6), (2,5), (2,6), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,5), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,7), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,3), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,0), (2,2), (2,2), (2,2), (2,0), (2,2), (2,0), (2,7), (2,7), (2,7), ...

In [ ]:
(
  mei.nmiSQRT,
  mei.f1,
  mei.rand,
  mei.arand
)

res34: (Double, Double, Double, Double) = (0.06054073807627623,0.25799546961688713,0.6355400899128606,0.020905379631695573)


(0.06054073807627623,0.25799546961688713,0.6355400899128606,0.020905379631695573)